# Project 2

# Exploring the 311 service request

We continued to use the 311 service request for analyzing. This time I chose to add 'Rat‑Related 311 Service Requests'to the project which specifically focuses on rat-related complaints via 311. Let's see the relationship between them.

# Step 1
install the plotly for visualization and get prepared with the datasets. 

In [2]:
%pip install plotly


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import plotly.express as px

url_all = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$limit=50000"
url_rat = "https://data.cityofnewyork.us/resource/cbqt-vkjz.csv?$limit=50000"

df_all = pd.read_csv(url_all, parse_dates=["created_date"])
df_rat = pd.read_csv(url_rat, parse_dates=["created_date"])

/var/folders/bj/gpmqq0dx5ld8w1w42kl86m2h0000gn/T/ipykernel_79363/3139748308.py:7: DtypeWarning: Columns (20,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(url_all, parse_dates=["created_date"])


# Step 2 
Cleaning the data and aggregate by day + borough

In [4]:
# create a 'day' column (date only)
df_all["day"] = df_all["created_date"].dt.to_period("D")
df_rat["day"] = df_rat["created_date"].dt.to_period("D")

print(df_all)
print(df_rat)

# aggregate by day + borough
agg_all = df_all.groupby(["borough", "day"]).size().reset_index(name="total_requests")
agg_rat = df_rat.groupby(["borough", "day"]).size().reset_index(name="rat_requests")

       unique_key        created_date              closed_date agency  \
0        66899990 2025-11-21 02:50:56                      NaN    DOT   
1        66900129 2025-11-21 02:20:43                      NaN   NYPD   
2        66901607 2025-11-21 02:20:25                      NaN   NYPD   
3        66903018 2025-11-21 02:20:18                      NaN   NYPD   
4        66897203 2025-11-21 02:19:44                      NaN   NYPD   
...           ...                 ...                      ...    ...   
49995    66862919 2025-11-16 09:45:00  2025-11-17T15:40:00.000   DSNY   
49996    66854542 2025-11-16 09:44:00  2025-11-16T09:51:00.000    DEP   
49997    66857819 2025-11-16 09:44:00                      NaN    DEP   
49998    66853488 2025-11-16 09:44:00                      NaN    DEP   
49999    66859070 2025-11-16 09:43:54  2025-11-16T14:04:34.000   NYPD   

                                  agency_name       complaint_type  \
0                Department of Transportation     Str

# Step 3
Merge and compute ratio, then convert Period to real datetime for Plotly.

In [5]:

df_merge = pd.merge(agg_all, agg_rat, on=["borough", "day"], how="left")
df_merge["rat_requests"] = df_merge["rat_requests"].fillna(0)
df_merge["rat_ratio"] = df_merge["rat_requests"] / df_merge["total_requests"]

df_merge["day"] = df_merge["day"].dt.to_timestamp()

In [6]:
df_all["created_date"].min(), df_all["created_date"].max(), df_rat["created_date"].min(), df_rat["created_date"].max()

(Timestamp('2025-11-16 09:43:54'),
 Timestamp('2025-11-21 02:50:56'),
 Timestamp('2025-09-12 15:59:48'),
 Timestamp('2025-11-21 02:15:51'))

# Step 4
Show the final result of the visualization.

In [11]:
%pip install kaleido


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
fig = px.line(
    df_merge,
    x="day",
    y="rat_ratio",
    color="borough",
    title="Rat Complaints as % of All 311 Requests (by Borough, Over Days)",
    labels={
        "day": "Day",
        "rat_ratio": "Rat Requests / Total 311 Requests",
        "borough": "Borough"
    }
)
fig.write_image("rat_ratio.png")  
fig.show()

ValueError: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido


# Conclusion
The rat-related complaints consistently make up a measurable share of 311 activity, with Manhattan and Brooklyn showing the highest ratios.
This implies that rodent issues are strongly associated with dense urban areas, older building stock, and high commercial activity—patterns confirmed through the relationship between these two datasets.